In [1]:
%%capture
%pip install llama-index qdrant-client llama-index-vector-stores-qdrant llama-index-llms-cohere llama-index-embeddings-cohere

In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

In [3]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [ ]:
#OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [4]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [5]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

# Query Pipelines

<img src="https://docs.llamaindex.ai/en/stable/_static/query/pipeline_rag_example.png">

Source: [LlamaIndex Docs](https://docs.llamaindex.ai/en/stable/module_guides/querying/pipeline/)

LlamaIndex offers a query API for chaining modules to manage data workflows easily. It revolves around the QueryPipeline, where you link various modules like LLMs, prompts, and retrievers in a sequence or DAG for end-to-end execution.

You can streamline workflows efficiently using QueryPipeline, reducing code complexity and enhancing readability. Additionally, a declarative interface ensures easy serialization of pipeline components for portability and deployment across systems in the future.

In [6]:
from llama_index.core.settings import Settings
from llama_index.llms.cohere import Cohere
from llama_index.embeddings.cohere import CohereEmbedding
Settings.llm = Cohere(model="command-r-plus", api_key=CO_API_KEY)
Settings.embed_model = CohereEmbedding(model_name="embed-english-v3.0", api_key=CO_API_KEY)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/llama/lib/python3.13/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
from qdrant_client import QdrantClient
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

# Create a Qdrant client
client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

# Create a Qdrant vector store
vector_store = QdrantVectorStore(
    client=client, 
    collection_name="it_can_be_done"
    )

# Create a vector store index
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=Settings.embed_model,
)

# A RAG Pipeline with PromptTemplate

I'm going to kick it off with a slightly complex workflow where the input is passes through two prompts before initiating retrieval.

1. Retrieve question about given topic.

2. Rephrase the context

Each prompt only takes in one input, so `QueryPipeline` will automatically chain LLM outputs into the prompt and then into the LLM.

You'll see how to define links more explicitly in the next section.

In [8]:
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate
from llama_index.core import PromptTemplate

# generate question regarding topic
prompt_str1 = "Retrieve context about the following topic: {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)

prompt_str2 = """Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

{query_str}

Syntesized response:
"""

prompt_tmpl2 = PromptTemplate(prompt_str2)

retriever = index.as_retriever(similarity_top_k=5)

p = QueryPipeline(
    chain=[
        prompt_tmpl1, 
        retriever,
        prompt_tmpl2, 
        Settings.llm, 
        ], 
        verbose=True
)

In [9]:
response = p.run(topic="Working hard to achieve your goals even when you doubt yourself and your chances of success")

> Running module 2307a7b3-a865-4b41-be15-ad8519789d9c with input: 
topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module 3b35da88-3195-42f8-bdb3-98aa0e7eba4a with input: 
input: Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module a110c258-8ab9-4918-af1f-82220e5ae26e with input: 
query_str: [NodeWithScore(node=TextNode(id_='d1707830-b877-4e2b-aecb-4942766991ac', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt',...

> Running module 5534a9b2-ec91-4515-a78e-485c86b11594 with input: 
messages: Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

['He knew \'twas time to put his grouch away upon the shelf.\r\n  And so he did.--You see,...



In [10]:
print(response)

assistant: So, like, this dude Griffith Alexander was like, "Yo, I was just talkin' to myself over here, no biggie." And then S.E. Kiser comes in hot with some real talk about how life ain't no walk in the park and we gotta stay strong and fight the good fight. It's like, we all gonna face some struggles, but it's all about keepin' that grind mentality and not lettin' fear and doubt hold us back. 

Then Margaret Widdemer hits us with some deep thoughts about pain and growth, like, "Yo, we all gonna experience pain, but it's all good 'cause it helps us relate to others and build that empathy." It's all about pickin' yourself back up and keepin' that hope alive. 

Grantland Rice ain't playin' around when he spits that fire about the power of positive thinkin'. He's like, "The word 'can't' is straight-up toxic, ya feel me? It's holdin' us back and crushin' our dreams." We gotta believe in ourselves and push through the hard times, 'cause that's how we level up in life. 

Etta Wheeler Wilc

You can debug the pipeline by viewing intermediate inputs and outputs

In [11]:
output, intermediates  = p.run_with_intermediates(topic="Working hard to achieve your goals even when you doubt yourself and your chances of success")

> Running module 2307a7b3-a865-4b41-be15-ad8519789d9c with input: 
topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module 3b35da88-3195-42f8-bdb3-98aa0e7eba4a with input: 
input: Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module a110c258-8ab9-4918-af1f-82220e5ae26e with input: 
query_str: [NodeWithScore(node=TextNode(id_='d1707830-b877-4e2b-aecb-4942766991ac', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt',...

> Running module 5534a9b2-ec91-4515-a78e-485c86b11594 with input: 
messages: Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

['He knew \'twas time to put his grouch away upon the shelf.\r\n  And so he did.--You see,...



In [12]:
output.__dict__

{'message': ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='So, like, this dude Griffith Alexander is basically saying that it\'s time to stop being a grouch and just move on. And then there\'s this poem by S.E. Kiser, "The Fighter," which is all about how life is a constant struggle and we gotta stay strong and keep fighting if we wanna come out on top. Margaret Widdemer hits us with some real talk in "To Youth After Pain," reminding us that pain is inevitable, but it\'s also what helps us relate to others and build sympathy.\n\nThen Grantland Rice comes through with "Can\'t," a poem that\'s like, "Yo, don\'t ever say you can\'t because that\'s when you fail." It\'s all about having the right mindset and not letting fear hold you back. And Etta Wheeler Wilcox is like, "Yeah, we all make mistakes, but that\'s how we learn and grow, so keep moving forward."\n\nWhistler and Shakespeare drop some knowledge about not letting gloom and negativity bring you down. It\'s all ab

In [13]:
intermediates

{'2307a7b3-a865-4b41-be15-ad8519789d9c': ComponentIntermediates(inputs={'topic': 'Working hard to achieve your goals even when you doubt yourself and your chances of success'}, outputs={'prompt': 'Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success'}),
 '3b35da88-3195-42f8-bdb3-98aa0e7eba4a': ComponentIntermediates(inputs={'input': 'Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success'}, outputs={'output': [NodeWithScore(node=TextNode(id_='d1707830-b877-4e2b-aecb-4942766991ac', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt', 'file_type': 'text/plain', 'file_size': 405245, 'creation_date': '2025-02-02', 'last_modified_date': '2024-11-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date'

In [15]:
#intermediates['d7612067-3809-4d48-aa49-0c957da8de40']
intermediates['2307a7b3-a865-4b41-be15-ad8519789d9c']

ComponentIntermediates(inputs={'topic': 'Working hard to achieve your goals even when you doubt yourself and your chances of success'}, outputs={'prompt': 'Retrieve context about the following topic: Working hard to achieve your goals even when you doubt yourself and your chances of success'})

### Another RAG Pipeline

Here we setup a RAG pipeline without the query rewriting step.

Here we need a way to link the input query to both the retriever and summarizer. 

We can do this by defining a special `InputComponent`, allowing us to link the inputs to multiple downstream modules.

In [16]:
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.query_pipeline import InputComponent
from llama_index.llms.openai import OpenAI

input = InputComponent()

retriever = index.as_retriever(similarity_top_k=5)

#llm = OpenAI(model="gpt-4o")

#tree_summarizer = TreeSummarize(llm=llm)
tree_summarizer = TreeSummarize(Settings.llm)

In [17]:
p = QueryPipeline(verbose=True, show_progress=True)

p.add_modules(
    {
        "input": input,
        "retriever": retriever,
        "tree_summarizer": tree_summarizer,
    }
)
p.add_link("input", "retriever")
p.add_link("input", "tree_summarizer", dest_key="query_str")
p.add_link("retriever", "tree_summarizer", dest_key="nodes")

In [18]:
response = p.run(input="Working hard to achieve your goals even when you doubt yourself and your chances of success")

> Running module input with input: 
input: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module retriever with input: 
input: Working hard to achieve your goals even when you doubt yourself and your chances of success

> Running module tree_summarizer with input: 
query_str: Working hard to achieve your goals even when you doubt yourself and your chances of success
nodes: [NodeWithScore(node=TextNode(id_='d1707830-b877-4e2b-aecb-4942766991ac', embedding=None, metadata={'file_path': '../02_Fundamental_Concepts_in_LlamaIndex/data/pg10763.txt', 'file_name': 'pg10763.txt',...



In [19]:
print(str(response))

It is important to maintain a fighting spirit and not give up in the face of adversity. While it is normal to experience self-doubt and fear, it is crucial to keep going and not let these emotions hold you back. 

Remember that struggles and setbacks are a natural part of life, and they provide an opportunity to grow stronger and gain valuable lessons. Embrace the challenges you encounter, as they will shape you into a more resilient and determined person. 

Believe in yourself and your abilities, and don't let negative thoughts or fears hold you back. You can achieve anything if you put your mind to it and work hard towards your goals. It is also important to maintain a positive attitude and not be discouraged by setbacks. 

As Napoleon is quoted as saying, "Impossible is a word found only in the dictionary of fools." So, keep fighting, and don't let doubt stand in the way of your success!


In [20]:
response.__dict__

{'response': 'It is important to maintain a fighting spirit and not give up in the face of adversity. While it is normal to experience self-doubt and fear, it is crucial to keep going and not let these emotions hold you back. \n\nRemember that struggles and setbacks are a natural part of life, and they provide an opportunity to grow stronger and gain valuable lessons. Embrace the challenges you encounter, as they will shape you into a more resilient and determined person. \n\nBelieve in yourself and your abilities, and don\'t let negative thoughts or fears hold you back. You can achieve anything if you put your mind to it and work hard towards your goals. It is also important to maintain a positive attitude and not be discouraged by setbacks. \n\nAs Napoleon is quoted as saying, "Impossible is a word found only in the dictionary of fools." So, keep fighting, and don\'t let doubt stand in the way of your success!',
 'source_nodes': [NodeWithScore(node=TextNode(id_='d1707830-b877-4e2b-ae